# Resting-state ICA

## Initiate session

In [ ]:
PARTICIPANT_ID = ''  # e.g., 'id000'

In [ ]:
# imports
from scripts.visualization import ICAPlotsIPython
from scripts.constants import RESTING_STATE_DIR, TMP_DIR, ICA_DIR
from IPython.display import display, Image
import matplotlib
import matplotlib.pyplot as plt
from mne.io import Raw
from mne.preprocessing import ICA
import os

# session configuration
EEG_PATH = os.path.join(RESTING_STATE_DIR, f'{PARTICIPANT_ID}_restingState_raw.fif')  # path to read raw EEG data from
ICA_OUTPUT_PATH = os.path.join(ICA_DIR, f'{PARTICIPANT_ID}-ica.fif')  # path to write ICA solution to
SRATE = 1000  # sampling rate in Hz
SCALE = 1e-6  # raw signal * SCALE = signal in volts
L_FREQ = .2  # lower passband edge for high-pass filtering
N_ICA_COMPONENTS = 32  # n ICA components to extract

In [ ]:
if not os.path.exists(ICA_DIR):
    os.makedirs(ICA_DIR)

## Read data

In [ ]:
eeg = Raw(EEG_PATH, preload=True, verbose=False)

In [ ]:
%matplotlib inline
_ = eeg.plot(
    n_channels=len(eeg.ch_names), 
    duration=5
)

## High-pass filter

In [ ]:
eeg = eeg.filter(
    l_freq=L_FREQ,
    h_freq=None,
    verbose=True
)

In [ ]:
_ = eeg.plot(
    n_channels=len(eeg.ch_names), 
    duration=5
)

## ICA

In [ ]:
%matplotlib inline

_ = plt.plot()
plt.close()

ica = ICA(
    N_ICA_COMPONENTS,
    max_iter='auto', 
    random_state=0
)

ica.fit(eeg, verbose='INFO')

In [ ]:
ICAPlotsIPython(
    ica=ica, 
    source=eeg,
    samples_window=[0, 3000],
    exclusive_components=None,
    output_dir=TMP_DIR
);

In [ ]:
for component in range(N_ICA_COMPONENTS):
    display(Image(filename=os.path.join(TMP_DIR, f'_{component}.png')))

In [ ]:
# keep unmodified copy until final component selection
_eeg = eeg.copy()

In [ ]:
# Step 1: select components to exclude
eeg = _eeg.copy()
ica.exclude = []  # e.g. [0, 1] to exclude components #0 and #1
eeg = ica.apply(eeg, verbose=False)

In [ ]:
# Step 2: plot time data after component exclusion
%matplotlib notebook
_ = eeg.plot(
    n_channels=len(eeg.ch_names), 
    duration=5,
)

In [ ]:
# if not satisfied and selection to be modified --> redo steps 1 & 2
del eeg

In [ ]:
# if satisfied and final component selection
del _eeg

In [ ]:
# save solution
ica.save(ICA_OUTPUT_PATH)

## Clear session

In [ ]:
tmp_files = [os.path.join(TMP_DIR, f) for f in os.listdir(TMP_DIR)]
for f in tmp_files:
    os.remove(f)